In [1]:
%autosave 10

Autosaving every 10 seconds


In [ ]:
#Testing the autosave function

In [3]:
#Now this is testing the automatic github pushing, is this working?

In [ ]:
#Yes it is! Yay!

# Continuing ML Tutorials and learning how to autosave on Github

## (That is what the above functions are showing)